In [1]:
from datasets import load_dataset

data_files = {"train" : "train_dataset.json", "validation" : "valid_dataset.json"}

dataset = load_dataset("json", data_files=data_files)

In [2]:
dataset

DatasetDict({
    train: Dataset({
        features: ['post_id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 16118
    })
    validation: Dataset({
        features: ['post_id', 'annotators', 'rationales', 'post_tokens'],
        num_rows: 8950
    })
})

In [3]:
def get_label(sample):
    #get annotator 
    annotators = sample['annotators']
    
    #get labels 
    label_dict = {}
    for annotator in annotators:
        if annotator['label'] not in label_dict.keys():
            label_dict[annotator['label']] = 1
        else:
            label_dict[annotator['label']] += 1
            
            
    #get final label         
    max_label = max(label_dict.values())
    res = list(filter(lambda x: label_dict[x] == max_label, label_dict))
    
    return {"final_label": res[0]}

In [4]:
dataset = dataset.map(get_label)

In [5]:
dataset

DatasetDict({
    train: Dataset({
        features: ['post_id', 'annotators', 'rationales', 'post_tokens', 'final_label'],
        num_rows: 16118
    })
    validation: Dataset({
        features: ['post_id', 'annotators', 'rationales', 'post_tokens', 'final_label'],
        num_rows: 8950
    })
})

In [6]:
post_tokens_train = dataset['train']['post_tokens']
post_tokens_test = dataset['validation']['post_tokens']
posts_train = [" ".join(post_token) for post_token in post_tokens_train]
posts_test = [" ".join(post_token) for post_token in post_tokens_test]

posts = posts_train + posts_test

In [7]:
labels_train = dataset['train']['final_label']
labels_test = dataset['validation']['final_label']

labels = labels_train + labels_test

In [8]:
print(len(posts), len(labels))

25068 25068


In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorizer = TfidfVectorizer()
vectorizer = vectorizer.fit(posts)
post_vectors = vectorizer.transform(posts)

In [10]:
from sklearn.preprocessing import LabelEncoder

enc = LabelEncoder()
encoded_labels = enc.fit_transform(labels)

In [11]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(post_vectors, encoded_labels, test_size = 0.2)

In [12]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(n_estimators = 500, random_state=42)
clf.fit(x_train, y_train)


RandomForestClassifier(n_estimators=500, random_state=42)

In [13]:
from sklearn.metrics import f1_score

y_pred = clf.predict(x_test)
f1_score = f1_score(y_test, y_pred, average = 'micro')

print(f1_score)

0.7415237335460709


In [19]:
import joblib

joblib.dump(vectorizer, "random_forest_vectorizer.joblib")

['random_forest_vectorizer.joblib']

In [20]:
import joblib

joblib.dump(enc, "random_forest_encoder.joblib")

['random_forest_encoder.joblib']

In [13]:
import joblib

#joblib.dump(clf, "random_forest_hatespeech.joblib")
loaded_clf = joblib.load("random_forest_hatespeech.joblib")

y_pred = loaded_clf.predict(x_test)

In [17]:
type(y_pred)

numpy.ndarray

In [16]:
type(y_test)

numpy.ndarray

In [18]:
from sklearn.metrics import f1_score
f1_score = f1_score(y_test, y_pred, average = 'micro')

print(f1_score)

0.9503390506581572


## Explaining with LIME

In [ ]:
list(enc.classes_)

In [ ]:
from lime import lime_text
from sklearn.pipeline import make_pipeline

c = make_pipeline(vectorizer, clf)

In [ ]:
posts[-1]

In [ ]:
labels[-1]

In [ ]:
print(c.predict_proba([posts[-1]]))

In [ ]:
from lime.lime_text import LimeTextExplainer
explainer = LimeTextExplainer(class_names = list(enc.classes_))

In [ ]:
exp = explainer.explain_instance(posts[-1], c.predict_proba)

In [ ]:
exp.as_list()

In [ ]:
%matplotlib inline
fig = exp.as_pyplot_figure()

In [ ]:
exp.show_in_notebook(text=False)

In [ ]:
exp.show_in_notebook(text=True)